<a href="https://colab.research.google.com/github/abn15/E-health-monitoring-system/blob/main/bandpass_notch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyEDFlib
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 59.1 MB/s eta 0:00:00


In [ ]:
!pip install -U mne
!pip install pywavelets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.2 MB/s eta 0:00:00


In [ ]:
import mne
import pywt
import pyedflib
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, savgol_filter

def apply_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_data = filtfilt(b, a, data)
    return filtered_data

def apply_notch_filter(data, freq, fs, Q=30):
    nyquist = 0.5 * fs
    notch = freq / nyquist
    b, a = butter(2, [notch - 1 / Q, notch + 1 / Q], btype='bandstop')
    filtered_data = filtfilt(b, a, data)
    return filtered_data

edf_file = pyedflib.EdfReader('/content/s01.edf')

num_channels = edf_file.signals_in_file
channel_data = [edf_file.readSignal(i) for i in range(num_channels)]

fs = edf_file.getSampleFrequency(0)

start_time = 0
end_time = 60
start_sample = int(start_time * fs)
end_sample = int(end_time * fs)
channel_data = [channel[start_sample:end_sample] for channel in channel_data]

modified_data = []
for channel_index in range(num_channels):
    data = channel_data[channel_index]

    filtered_data = apply_bandpass_filter(data, 0.1, 70, fs)

    filtered_data = apply_notch_filter(filtered_data, 50, fs)

    modified_data.append(filtered_data)

edf_file.close()

new=[]
new=savgol_filter(modified_data,31,2)

combined=[]
for channel in new:
    combined =[combined channel]

'''
new=np.array(new)
#new_dfs = [pd.DataFrame(data, columns=[f'preprocessed data of channel {i+1}']) for i, data in enumerate(new)]
new_dfs=[pd.DataFrame(data)]
output_file = 's01_preprocessed.xlsx'
#writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
new_dfs.to_excel(output_file,index=False)
'''
for i, data in enumerate(new_dfs):
    data.to_excel(writer, sheet_name=f'preprocessed data of channel {i+1}',columns =i+1, index=False)

    writer.save()

#print("Modified Data :\n",new)
decomposition_level = 5
wavelet = 'sym9'

# Apply the DWT decomposition to each EEG channel
processed_data=[]
for channel in channel_data:
    coeffs = pywt.wavedec(channel, wavelet, level=decomposition_level)

    # Perform processing on the coefficients (e.g., denoising, feature extraction)

    # Reconstruct the channel using the inverse DWT
    reconstructed_channel = pywt.waverec(coeffs, wavelet)

    processed_data.append(reconstructed_channel)
    approximation_coeffs = coeffs[0]
    detail_coeffs = coeffs[1:]

# Print the approximation and detail coefficients
    print("Approximation Coefficients:")
    print(approximation_coeffs)
    print()

    print("Detail Coefficients:")
    for level, detail_coeff in enumerate(detail_coeffs, start=1):
        print(f"Level {level}:")
        print(detail_coeff)
        print()
    approximation_df = pd.DataFrame(approximation_coeffs, columns=['Approximation Coefficients'])
    detail_dfs = [pd.DataFrame(detail_coeff, columns=[f'Detail Coefficients Level {i+1}']) for i, detail_coeff in enumerate(detail_coeffs)]

    output_file = 'coefficients.xlsx'
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

    approximation_df.to_excel(writer, sheet_name='Approximation Coefficients', index=False)
    for i, detail_df in enumerate(detail_dfs):
        detail_df.to_excel(writer, sheet_name=f'Detail Coefficients Level {i+1}', index=False)

    writer.save()
# Convert the processed data back to a numpy array
processed_data = np.array(processed_data)

#print('Processed EEG Data:', processed_data)
time = np.arange(start_time, end_time, 1 / fs)

plt.figure(figsize=(20, 30))
for channel_index in range(num_channels):
    plt.subplot(num_channels, 1, channel_index+1)
    plt.plot(time, channel_data[channel_index], label='Original')
    plt.plot(time, new[channel_index], label='savgol')
    plt.plot(time, modified_data[channel_index], label='modified')
    #plt.plot(time,processed_data[channel_index],label='DWT')
    plt.title('Channel {}'.format(channel_index+1))
    plt.xlabel('Time')
    plt.ylabel('Signal')
    plt.legend()

plt.tight_layout()
plt.show()'''

AttributeError: ignored

In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 12.9 MB/s eta 0:00:00
